# Problema 3: Comparación de Rendimiento con GLPK/Pyomo
## 1. Modele el problema en Pyomo utilizando el solver GLPK.

In [ ]:
from __future__ import division
from pyomo.environ import *

from pyomo.opt import SolverFactory

Model = ConcreteModel()
# Data de entrada
maxC=10
maxB=8

# Conjuntos
D=RangeSet(1, maxC)
O=RangeSet(1, maxB)

# Parametros
c = {1:5, 2:8, 3:3, 4:7, 5:6, 6:9, 7:4, 8:10, 9:2, 10:11}
b = {1:50, 2:60, 3:55, 4:40, 5:45, 6:70, 7:65, 8:50}

a1= {1:1, 2:2, 3:1, 4:1, 5:0, 6:0, 7:3, 8:1, 9:2, 10:1}
a2= {1:2, 2:1, 3:0, 4:2, 5:1, 6:1, 7:0, 8:3, 9:1, 10:2}
a3= {1:1, 2:1, 3:2, 4:0, 5:2, 6:1, 7:1, 8:0, 9:3, 10:1}
a4= {1:0, 2:2, 3:1, 4:1, 5:1, 6:0, 7:2, 8:1, 9:1, 10:1}
a5= {1:2, 2:0, 3:1, 4:1, 5:1, 6:2, 7:1, 8:1, 9:0, 10:2}
a6= {1:1, 2:1, 3:1, 4:1, 5:1, 6:1, 7:1, 8:1, 9:1, 10:1}
a7= {1:0, 2:1, 3:2, 4:1, 5:0, 6:1, 7:2, 8:1, 9:1, 10:0}
a8= {1:1, 2:0, 3:1, 4:2, 5:1, 6:0, 7:1, 8:2, 9:1, 10:1}
a= {1: a1, 2: a2, 3: a3, 4: a4, 5: a5, 6: a6, 7: a7, 8: a8}

# Variable de decisión
Model.x = Var(D, domain=NonNegativeReals)

# función objetivo
Model.obj = Objective(expr=sum(c[i]*Model.x[i] for i in D), sense=maximize)

# Restricciones
# rest. No. 27
Model.rest1 = ConstraintList()
for j in O:
    Model.rest1.add(expr = sum(Model.x[i]*a[j][i] for i in D) <= b[j])

# rest. No. 28
Model.rest2 = ConstraintList()
for i in D:
    Model.rest2.add(expr = Model.x[i] >= 0)
    
# Especificación del solver
SolverFactory('glpk').solve(Model)

Model.display()

Model unknown

  Variables:
    x : Size=10, Index=x_index
        Key : Lower : Value  : Upper : Fixed : Stale : Domain
          1 :     0 :    0.0 :  None : False : False : NonNegativeReals
          2 :     0 : 15.625 :  None : False : False : NonNegativeReals
          3 :     0 :    0.0 :  None : False : False : NonNegativeReals
          4 :     0 :    0.0 :  None : False : False : NonNegativeReals
          5 :     0 :    0.0 :  None : False : False : NonNegativeReals
          6 :     0 : 18.125 :  None : False : False : NonNegativeReals
          7 :     0 :    0.0 :  None : False : False : NonNegativeReals
          8 :     0 :   8.75 :  None : False : False : NonNegativeReals
          9 :     0 :    0.0 :  None : False : False : NonNegativeReals
         10 :     0 :    0.0 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 375.625

  Constraints:
    rest1 : Size=8
     

In [15]:
# resultados
print("\n### Resumen de la Solución ###")
print("Valor óptimo de la función objetivo:", value(Model.obj))
print("Componentes seleccionados:")

for i in D:
    if value(Model.x[i]) >= 0.99:  # si es 1
        print("  - Componente {} (costo: {}, valor de x[{}]:{})".format(i, c[i],i, value(Model.x[i])))



### Resumen de la Solución ###
Valor óptimo de la función objetivo: 375.625
Componentes seleccionados:
  - Componente 2 (costo: 8, valor de x[2]:15.625)
  - Componente 6 (costo: 9, valor de x[6]:18.125)
  - Componente 8 (costo: 10, valor de x[8]:8.75)


## 2. Modelar el problema en Simplex
### Nuevas definiciones del problema
#### Función a maximizar: 
$Z = 5*x_{1}+8*x_{2}+3*x_{3}+7*x_{4}+6*x_{5}+9*x_{6}+4*x_{7}+10*x_{8}+2*x_{9}+11*x_{10}$

#### Restricciones/ Sujeto a :
- $1*x_{1}+2*x_{2}+1*x_{3}+1*x_{4}+0*x_{5}+0*x_{6}+3*x_{7}+1*x_{8}+2*x_{9}+1*x_{10}  \leq 50$
- $2*x_{1}+1*x_{2}+0*x_{3}+2*x_{4}+1*x_{5}+1*x_{6}+0*x_{7}+3*x_{8}+1*x_{9}+2*x_{10}  \leq 60$
- $1*x_{1}+1*x_{2}+2*x_{3}+0*x_{4}+2*x_{5}+1*x_{6}+1*x_{7}+0*x_{8}+3*x_{9}+1*x_{10}  \leq 55$
- $0*x_{1}+2*x_{2}+1*x_{3}+1*x_{4}+1*x_{5}+0*x_{6}+2*x_{7}+1*x_{8}+1*x_{9}+1*x_{10}  \leq 40$
- $2*x_{1}+0*x_{2}+1*x_{3}+1*x_{4}+1*x_{5}+2*x_{6}+1*x_{7}+1*x_{8}+0*x_{9}+2*x_{10}  \leq 45$
- $1*x_{1}+1*x_{2}+1*x_{3}+1*x_{4}+1*x_{5}+1*x_{6}+1*x_{7}+1*x_{8}+1*x_{9}+1*x_{10}  \leq 70$
- $0*x_{1}+1*x_{2}+2*x_{3}+1*x_{4}+0*x_{5}+1*x_{6}+2*x_{7}+1*x_{8}+1*x_{9}+0*x_{10}  \leq 65$
- $1*x_{1}+0*x_{2}+1*x_{3}+2*x_{4}+1*x_{5}+0*x_{6}+1*x_{7}+2*x_{8}+1*x_{9}+1*x_{10}  \leq 50$
- Que todas las $x_{i}$ sean mayores o iguales a 0



### 2.1 Simplex estándar

In [19]:
from tabulate import tabulate


def mostrarTabla(M, funcion, iteracion, n_var, n_eq):
    headers = (
        [f"x{i+1}" for i in range(n_var)] + [f"s{i+1}" for i in range(n_eq)] + ["b"]
    )
    print("Iteracion #" + str(iteracion))
    print(tabulate(M, headers=headers, tablefmt="fancy_grid"))
    print("Función objetivo:")
    print(tabulate([funcion], headers=headers, tablefmt="fancy_grid"))


def simplex(f, A, b):
    n_var = len(f)
    n_eq = len(b)

    M = []
    for i in range(n_eq):
        row = A[i] + [0] * n_eq + [b[i]]
        row[n_var + i] = 1
        M.append(row)

    funcion = [-x for x in f] + [0] * n_eq + [0]

    it = 1
    while True:
        index_to_loose = 0
        for j in range(1, len(funcion) - 1):
            if funcion[j] < funcion[index_to_loose]:
                index_to_loose = j

        if funcion[index_to_loose] >= 0:
            break

        index_to_tighten = None
        valueMin = 1e9
        for i in range(n_eq):
            if M[i][index_to_loose] > 0:
                if M[i][-1] / M[i][index_to_loose] < valueMin:
                    valueMin = M[i][-1] / M[i][index_to_loose]
                    index_to_tighten = i

        pivot_val = M[index_to_tighten][index_to_loose]
        M[index_to_tighten] = [x / pivot_val for x in M[index_to_tighten]]

        for i in range(n_eq):
            if i != index_to_tighten:
                ratio = M[i][index_to_loose]
                M[i] = [
                    M[i][j] - ratio * M[index_to_tighten][j] for j in range(len(M[0]))
                ]

        ratio = funcion[index_to_loose]
        funcion = [
            funcion[j] - ratio * M[index_to_tighten][j] for j in range(len(funcion))
        ]

        mostrarTabla(M, funcion, it, n_var, n_eq)
        it += 1

    sol = [0] * n_var
    for j in range(n_var):
        fila_donde_hay_var = -1

        for i in range(n_eq):
            if M[i][j] == 1 and all(M[k][j] == 0 for k in range(n_eq) if k != i):
                fila_donde_hay_var = i
                break

        if fila_donde_hay_var != -1:
            sol[j] = M[fila_donde_hay_var][-1]  # igual a valor libre

    print("Maximo: " + str(funcion[-1]))

    for i in range(n_var):
        print("x[" + str(i + 1) + "] = " + str(sol[i]))


# Para el presente programa se considera la ecuación Ax <= b.
# f representa los coeficientes de la funcion a maximizar
# de forma que el producto interno entre f y x, da el valor maximo.

# La implementación está basada en la explicación de simplex presente en
# https://www.youtube.com/watch?v=E72DWgKP_1Y
# por lo que los nombres de variables corresponden a tight/loose en lugar de
# basicas (y no), obedeciendo a nombres representativos frente a la interpretacion
# geometrica conversada en clase.

f = list(c.values())

A = [list(a1.values()), list(a2.values()), list(a3.values()), list(a4.values()), list(a5.values()), list(a6.values()), list(a7.values()), list(a8.values())]

b2 = list(b.values())

simplex(f, A, b2)


Iteracion #1
╒══════╤══════╤══════╤══════╤══════╤══════╤══════╤══════╤══════╤═══════╤══════╤══════╤══════╤══════╤══════╤══════╤══════╤══════╤══════╕
│   x1 │   x2 │   x3 │   x4 │   x5 │   x6 │   x7 │   x8 │   x9 │   x10 │   s1 │   s2 │   s3 │   s4 │   s5 │   s6 │   s7 │   s8 │    b │
╞══════╪══════╪══════╪══════╪══════╪══════╪══════╪══════╪══════╪═══════╪══════╪══════╪══════╪══════╪══════╪══════╪══════╪══════╪══════╡
│    0 │    2 │  0.5 │  0.5 │ -0.5 │   -1 │  2.5 │  0.5 │    2 │     0 │    1 │    0 │    0 │    0 │ -0.5 │    0 │    0 │    0 │ 27.5 │
├──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┤
│    0 │    1 │ -1   │  1   │  0   │   -1 │ -1   │  2   │    1 │     0 │    0 │    1 │    0 │    0 │ -1   │    0 │    0 │    0 │ 15   │
├──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┤
│    0 │    1 │  1.5 │ -0.5 │  1.5 

### 2.2 Simplex Dual Phase

In [23]:
import numpy as np
import pandas as pd
def show_table(table: pd.DataFrame, title: str = ""):
    from tabulate import tabulate
    
    if title:
        print(f"=== {title} ===")
    print(tabulate(table, headers="keys", tablefmt="psql", showindex=True))
    print()  # Separación final

# Definición de la matriz de restricciones y lado derecho.
# Orden de columnas: las primeras 10 corresponden a las variables originales (x1 a x10),
# las siguientes 8 a las variables de holgura (s1 a s8) y finalmente el lado derecho 'ld'.
restricciones = np.array([
    # x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, s1, s2, s3, s4, s5, s6, s7, s8, ld
    [ 1, 2, 1, 1, 0, 0, 3, 1, 2,  1,   1, 0, 0, 0, 0, 0, 0, 0, 50 ],  # Restricción 1 (≤ 50)
    [ 2, 1, 0, 2, 1, 1, 0, 3, 1,  2,   0, 1, 0, 0, 0, 0, 0, 0, 60 ],  # Restricción 2 (≤ 60)
    [ 1, 1, 2, 0, 2, 1, 1, 0, 3,  1,   0, 0, 1, 0, 0, 0, 0, 0, 55 ],  # Restricción 3 (≤ 55)
    [ 0, 2, 1, 1, 1, 0, 2, 1, 1,  1,   0, 0, 0, 1, 0, 0, 0, 0, 40 ],  # Restricción 4 (≤ 40)
    [ 2, 0, 1, 1, 1, 2, 1, 1, 0,  2,   0, 0, 0, 0, 1, 0, 0, 0, 45 ],  # Restricción 5 (≤ 45)
    [ 1, 1, 1, 1, 1, 1, 1, 1, 1,  1,   0, 0, 0, 0, 0, 1, 0, 0, 70 ],  # Restricción 6 (≤ 70)
    [ 0, 1, 2, 1, 0, 1, 2, 1, 1,  0,   0, 0, 0, 0, 0, 0, 1, 0, 65 ],  # Restricción 7 (≤ 65)
    [ 1, 0, 1, 2, 1, 0, 1, 2, 1,  1,   0, 0, 0, 0, 0, 0, 0, 1, 50 ]   # Restricción 8 (≤ 50)
])

# Nombres de columnas (variables originales, de holgura y lado derecho)
columnas = ['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10',
            's1','s2','s3','s4','s5','s6','s7','s8','ld']

# Variables básicas iniciales: las variables de holgura
base = ['s1','s2','s3','s4','s5','s6','s7','s8']

# Coeficientes de la función objetivo
# Función a maximizar: 
#     Z = 5*x1 + 8*x2 + 3*x3 + 7*x4 + 6*x5 + 9*x6 + 4*x7 + 10*x8 + 2*x9 + 11*x10
# En la tabla simplex se coloca la fila Z con los coeficientes multiplicados por (-1).
coeficientes_z = {
    'x1': -5, 'x2': -8, 'x3': -3, 'x4': -7, 'x5': -6,
    'x6': -9, 'x7': -4, 'x8': -10, 'x9': -2, 'x10': -11,
    's1': 0, 's2': 0, 's3': 0, 's4': 0, 's5': 0, 's6': 0, 's7': 0, 's8': 0,
    'ld': 0
}

# Armado de la tabla inicial utilizando pandas
tabla_inicial = pd.DataFrame(restricciones, columns=columnas, index=base)

# Agregar la fila de la función objetivo Z
fila_z = [ coeficientes_z[var] for var in columnas ]
tabla_inicial.loc['Z'] = fila_z

print("Tabla inicial (Simplex - Función a Maximizar):")
print(tabla_inicial)
def calcular_costos_reducidos(tabla, base):
    #sacar z de la tabla
    z = np.zeros(tabla.shape[1]) # inicializar la fila Z con ceros

    # multiplcar la fila de cada base por su coeficiente en la fila Z ej: x4 -> [2,1,-1,1,0,0,0,10] * 0
    for b in base:
        coef = tabla.loc['Z', b]
        fila = tabla.loc[b]
        z += coef * fila
    
    tabla.loc['Z'] = z  # actualizar fila Z con los costos reducidos
    return tabla

tabla_actualizada = calcular_costos_reducidos(tabla_inicial.copy(), base)
print("\nTabla inicial:")
show_table(tabla_actualizada)
def verificar_optimalidad(tabla):
    # Verificar si todos los costos reducidos son no negativos
    return all(tabla.loc['Z'][:-1] >= 0)  # Exepto el lado derecho

def seleccionar_var_entrada(tabla, base):
    z_values = tabla.loc['Z'][:-1]  # Excluyendo el lado derecho
    z_values = z_values.drop(labels=base, errors='ignore')  # ← evitar repetir variables de la base
    indice_mas_neg = np.argmin(z_values)  # Posicion del costo reducido más negativo
    valor = z_values.iloc[indice_mas_neg]  # Valor del costo reducido más negativo

    if valor >= 0:
        return None
    var_entrada = z_values.index[indice_mas_neg]
    print(f"Variable de entrada: {var_entrada} (costo reducido = {valor:.4f})")
    return var_entrada

def calcular_razones(tabla, var_entrada):
    # Calcular las razones para determinar la variable de salida
    razones = []
    filas_basicas = tabla.index[:-1]  # Excluyendo la fila Z
    for fila in filas_basicas:
        coef = tabla.loc[fila, var_entrada]
        ld = tabla.loc[fila, 'ld']
        if coef > 0:
            razon = ld / coef
            razones.append(razon)
        else:
            razones.append( np.inf)
    idx_var_salida = np.argmin(razones)
    var_salida = filas_basicas[idx_var_salida]
    print(f"Variable de salida: {var_salida} (razón mínima = {razones[idx_var_salida]:.4f})")
    return var_salida
def pivoteo(tabla, var_entrada, var_salida):
    nueva_tabla = tabla.copy()
    print(f"\nPivoteo: {var_entrada} entra, {var_salida} sale")
    pivote = nueva_tabla.loc[var_salida, var_entrada]  # Elemento pivote
    print(f"Elemento pivote: {pivote}")
    nueva_tabla.loc[var_salida] = nueva_tabla.loc[var_salida] / pivote  

    # volver 0 los demás elementos de la columna de la variable de entrada
    for fila in nueva_tabla.index:
        if fila != var_salida:
            coef = nueva_tabla.loc[fila, var_entrada]
            nueva_tabla.loc[fila] = nueva_tabla.loc[fila] - coef * nueva_tabla.loc[var_salida]
    
    # Actualizar la variable básica
    
    nueva_base =  []
    for fila in nueva_tabla.index[:-1]:
        if fila == var_salida:
            nueva_base.append(var_entrada)  # Cambiar la variable básica , reemplazar la variable que sale por la que entra
        else:
            nueva_base.append(fila) # Mantener la variable básica
    nueva_tabla.index = nueva_base + ['Z']  # Actualizar el índice de la tabla
    return nueva_tabla, nueva_base  

def simplex(tabla, base):
    iteracion = 0

    while True:
        print(f"\n Iteración {iteracion}")
        tabla = calcular_costos_reducidos(tabla, base)  # Calcular costos reducidos

        print("Costos reducidos:")
        show_table(tabla) 

        if verificar_optimalidad(tabla):
            print("\n Solución óptima alcanzada (Fase 1 terminada).")
            break

        var_entrada = seleccionar_var_entrada(tabla, base)  # Seleccionar variable de entrada

        if var_entrada is None:
            print("No hay variable de entrada. Ha terminado el algoritmo.")
            break

        var_salida = calcular_razones(tabla, var_entrada)

        tabla, base = pivoteo(tabla, var_entrada, var_salida)  # Pivoteo actualiza tabla y base

        print(" Tabla después del pivoteo:")
        show_table(tabla) 
        iteracion += 1

    return tabla, base

tabla_inicial= tabla_inicial.astype(float)  
tabla_final, base_final = simplex(tabla_inicial.copy(), base.copy())

def eliminar_artificiales(tabla,base, artificiales):
    tabla_final = tabla.copy()
    # Eliminar variables artificiales de la tabla
    tabla_final = tabla_final.drop(artificiales, axis=1)
    base_final = [var for var in base if var not in artificiales]  # Actualizar la base eliminando variables artificiales

    return tabla_final, base_final

def restablecer_f_o(tabla, coeficientes_z):
    tabla.loc['Z'] = [coeficientes_z.get(col, 0) for col in tabla.columns]
    return tabla


## PARA IMPRIMIR


# Mostrar la tabla final obtenida tras la ejecución del algoritmo simplex
print("\nTabla final (Solución Óptima):")
show_table(tabla_final, title="Tabla Final Simplex")

# Extraer la solución óptima para las variables originales (x1 a x10)
solucion = {}
for var in ['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10']:
    # Si la variable está en la base, su valor es el del lado derecho (ld) de la fila correspondiente;
    # de lo contrario, al ser no básica, su valor es 0.
    if var in base_final:
        solucion[var] = tabla_final.loc[var, 'ld']
    else:
        solucion[var] = 0

# Imprimir la solución óptima
print("Solución óptima de las variables:")
for var, valor in solucion.items():
    print(f"  {var} = {valor}")

# Mostrar el valor óptimo de la función objetivo (se encuentra en la fila 'Z' en la columna 'ld')
print("\nValor óptimo de la función objetivo:", tabla_final.loc['Z', 'ld'])


Tabla inicial (Simplex - Función a Maximizar):
    x1  x2  x3  x4  x5  x6  x7  x8  x9  x10  s1  s2  s3  s4  s5  s6  s7  s8   
s1   1   2   1   1   0   0   3   1   2    1   1   0   0   0   0   0   0   0  \
s2   2   1   0   2   1   1   0   3   1    2   0   1   0   0   0   0   0   0   
s3   1   1   2   0   2   1   1   0   3    1   0   0   1   0   0   0   0   0   
s4   0   2   1   1   1   0   2   1   1    1   0   0   0   1   0   0   0   0   
s5   2   0   1   1   1   2   1   1   0    2   0   0   0   0   1   0   0   0   
s6   1   1   1   1   1   1   1   1   1    1   0   0   0   0   0   1   0   0   
s7   0   1   2   1   0   1   2   1   1    0   0   0   0   0   0   0   1   0   
s8   1   0   1   2   1   0   1   2   1    1   0   0   0   0   0   0   0   1   
Z   -5  -8  -3  -7  -6  -9  -4 -10  -2  -11   0   0   0   0   0   0   0   0   

    ld  
s1  50  
s2  60  
s3  55  
s4  40  
s5  45  
s6  70  
s7  65  
s8  50  
Z    0  

Tabla inicial:
+----+------+------+------+------+------+------+------+-